## Comparative test
Sample THEx data so that it is consistent with the filtered PLASTICCS data, and create another test set of data with the same number of samples in each class (Ia and II) but randomly sampled from our dataset. We train the KDE multiclass classifier using the same training set, and test on these two different test sets. We aggregate results over a certain number of runs and compare the resulting metrics to see if differently distributed data alters the performance of the model.

In [ ]:
from evaluation.sampling_test import *


In [ ]:
main()

### 'breaking it'

Following is code to 'break it', by training on low r mag and testing on high r mag

In [ ]:
import os
import random
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


from models.multi_model.multi_model import MultiModel
from evaluation.plotting import *
from estimate.get_data import *


from evaluation.sampling_test import *

ordered_mags = ["g_mag", "r_mag", "i_mag", "z_mag", "y_mag",
                "W1_mag", "W2_mag", "H_mag", "K_mag", 'J_mag']

exp = str(random.randint(1, 10**10))
output_dir = "../figures/evaluation/" + exp
os.mkdir(output_dir)

cols = ["g_mag", "r_mag", "i_mag", "z_mag", "y_mag",
        "W1_mag", "W2_mag", "H_mag", "K_mag", 'J_mag',
        'redshift']
model = MultiModel(cols=cols,
                   class_labels=['Unspecified Ia', 'Unspecified II'],
                   transform_features=False,
                   min_class_size=40
                   ) 
# thex_dataset = pd.concat([model.X, model.y], axis=1)


# Sample from set for training data
# X_test = model.X.sample(frac=0.2)
# y_test = model.y.iloc[X_test.index].reset_index(drop=True)
# X_test.reset_index(drop=True, inplace=True)

# new_X_train, new_y_train = remove_data(X_test, model.X, model.y)

rcutoff = 19
new_X_train = model.X.loc[model.X['r_mag'] < rcutoff]
new_y_train = model.y.iloc[new_X_train.index].reset_index(drop=True)
new_X_train.reset_index(drop=True, inplace=True)

X_test = model.X.loc[model.X['r_mag'] >= rcutoff]
y_test = model.y.iloc[X_test.index].reset_index(drop=True)
X_test.reset_index(drop=True, inplace=True)

# Train on low r mags
model.train_model(new_X_train[ordered_mags], new_y_train)

# Test on high r mags
results = []
probabilities = model.get_all_class_probabilities(X_test[ordered_mags], model.normalize)
# Add labels as column to probabilities, for later evaluation
label_column = y_test['transient_type'].values.reshape(-1, 1)
probabilities = np.hstack((probabilities, label_column))
results.append(probabilities)

 
ii_train = get_thex_class_data("Unspecified II", pd.concat([new_X_train, new_y_train], axis=1))

ii_test = get_thex_class_data("Unspecified II", pd.concat([X_test, y_test], axis=1))

plot_compare_feature_dists('r_mag', "Unspecified II", rand_sample=ii_train, sampled=ii_test)


ia_train = get_thex_class_data("Unspecified Ia", pd.concat([new_X_train, new_y_train], axis=1))

ia_test = get_thex_class_data("Unspecified Ia", pd.concat([X_test, y_test], axis=1))

plot_compare_feature_dists('r_mag', "Unspecified Ia", rand_sample=ia_train, sampled=ia_test)

plot_performance(model=model,
                 testdata_y=y_test, 
                 output_dir=output_dir + '/rtest', 
                 results=results)